In [ ]:
import matplotlib.pyplot as plt
import math, random
from scipy.interpolate import make_smoothing_spline

from typing import List, Tuple, Union
import numpy as np

## Test functions to generate random polygons

Taken from: https://stackoverflow.com/questions/8997099/algorithm-to-generate-random-2d-polygon

In [ ]:
def generate_polygon(
    center: Tuple[float, float],
    avg_radius: float,
    irregularity: float,
    spikiness: float,
    num_vertices: int,
) -> List[Tuple[float, float]]:
    """
    Start with the center of the polygon at center, then creates the
    polygon by sampling points on a circle around the center.
    Random noise is added by varying the angular spacing between
    sequential points, and by varying the radial distance of each
    point from the centre.

    Args:
        center (Tuple[float, float]):
            a pair representing the center of the circumference used
            to generate the polygon.
        avg_radius (float):
            the average radius (distance of each generated vertex to
            the center of the circumference) used to generate points
            with a normal distribution.
        irregularity (float):
            variance of the spacing of the angles between consecutive
            vertices.
        spikiness (float):
            variance of the distance of each vertex to the center of
            the circumference.
        num_vertices (int):
            the number of vertices of the polygon.
    Returns:
        List[Tuple[float, float]]: list of vertices, in CCW order.
    """
    # Parameter check
    if irregularity < 0 or irregularity > 1:
        raise ValueError("Irregularity must be between 0 and 1.")
    if spikiness < 0 or spikiness > 1:
        raise ValueError("Spikiness must be between 0 and 1.")

    irregularity *= 2 * math.pi / num_vertices
    spikiness *= avg_radius
    angle_steps = random_angle_steps(num_vertices, irregularity)

    # now generate the points
    points = []
    angle = random.uniform(0, 2 * math.pi)
    for i in range(num_vertices):
        radius = clip(random.gauss(avg_radius, spikiness), 0, 2 * avg_radius)
        point = (
            center[0] + radius * math.cos(angle),
            center[1] + radius * math.sin(angle),
        )
        points.append(point)
        angle += angle_steps[i]

    return points


def random_angle_steps(steps: int, irregularity: float) -> List[float]:
    """Generates the division of a circumference in random angles.

    Args:
        steps (int):
            the number of angles to generate.
        irregularity (float):
            variance of the spacing of the angles between consecutive vertices.
    Returns:
        List[float]: the list of the random angles.
    """
    # generate n angle steps
    angles = []
    lower = (2 * math.pi / steps) - irregularity
    upper = (2 * math.pi / steps) + irregularity
    cumsum = 0
    for i in range(steps):
        angle = random.uniform(lower, upper)
        angles.append(angle)
        cumsum += angle

    # normalize the steps so that point 0 and point n+1 are the samehttps://stackoverflow.com/questions/8997099/algorithm-to-generate-random-2d-polygon
    cumsum /= 2 * math.pi
    for i in range(steps):
        angles[i] /= cumsum
    return angles


def clip(value, lower, upper):
    """
    Given an interval, values outside the interval are clipped to the interval
    edges.
    """
    return min(upper, max(value, lower))


def in_polygon(xy, poly):
    n = len(poly)
    inside = False
    p2x = 0.0
    p2y = 0.0
    xints = 0.0
    p1x, p1y = poly[0]
    x, y = xy
    for i in range(n + 1):
        p2x, p2y = poly[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xints = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x, p1y = p2x, p2y

    return inside

In [ ]:
class FovPolygon:
    def __init__(self, boundary: np.ndarray):
        """Boundary for fov in cartesian coordinates"""
        self.boundary = boundary

    def boundary_wrapped(self) -> np.ndarray:
        """Returns boundary wrapped around"""
        return np.concatenate((self.boundary, self.boundary[0, :][:, None].T), axis=0)

    def boundary_polar(self) -> np.ndarray:
        """Convert boundary from cartesian to polar"""
        az = np.arctan2(self.boundary[:, 1], self.boundary[:, 0])
        rng = np.linalg.norm(self.boundary[:, :2], axis=1)
        return np.concatenate((rng[:, None], az[:, None]), axis=1)

    def boundary_spline_polar(self, smoothing: float = 0.25):
        boundary_polar = self.boundary_polar()
        idx_sort = boundary_polar[:, 1].argsort()  # az
        spline = make_smoothing_spline(
            boundary_polar[idx_sort, 1],  # az
            boundary_polar[idx_sort, 0],  # rng
            lam=smoothing,
        )
        return spline

    def check_point(self, pt_xy) -> bool:
        return in_polygon(pt_xy, self.boundary)


def random_fov(avg_radius: float) -> FovPolygon:
    """create a ground truth polygon"""
    polygon_gt = generate_polygon(
        center=(0, 0),
        avg_radius=avg_radius,
        irregularity=0.25,
        spikiness=0.4,
        num_vertices=20,
    )
    # Wrap into an fov polygon
    return FovPolygon(np.array(polygon_gt))


def random_a_b(a: float, b: float) -> float:
    """Randomly sample within bounds"""
    assert b > a
    return (b - a) * np.random.rand() + a

## PolyLidar Model and Wrapper

In [ ]:
from polylidar import MatrixDouble, Polylidar3D


class PolyLidar:
    """This will be replaced with whatever is in the polylidar API"""

    def __init__(self, lmax: float = 1.8, min_triangles: int = 30):
        self.polylidar = Polylidar3D(lmax=lmax, min_triangles=min_triangles)

    def __call__(self, point_cloud: np.ndarray) -> np.ndarray:
        points_mat = MatrixDouble(point_cloud, copy=False)
        mesh, _, polygons = self.polylidar.extract_planes_and_polygons(points_mat)
        shell_indices = [polygon.shell for polygon in polygons]
        vertices = np.asarray(mesh.vertices)

        # Assuming there is only one polygon shell for now
        polygon_vertices = [list(vertices[i]) for i in list(shell_indices[0])]
        return np.array(polygon_vertices)


class PolyLidarWrapper:
    """This will remain to eventually wrap the avstack inputs into polylidar"""

    def __init__(self, model: PolyLidar):
        self.model = model

    def __call__(self, point_cloud: np.ndarray) -> FovPolygon:
        return FovPolygon(self.model(point_cloud))

In [ ]:
# sample the polygon via rejection sampling
fov = random_fov(avg_radius=5)
n_points_needed = 600
points_sampled = []
sample_domain = [(-20, 20), (-20, 20)]
while len(points_sampled) < n_points_needed:
    candidate_point = [random_a_b(*sample_domain[0]), random_a_b(*sample_domain[1])]
    if fov.check_point(candidate_point):
        points_sampled.append(candidate_point)
points_sampled = np.array(points_sampled)

# run the model on the sampled points
model = PolyLidarWrapper(model=PolyLidar())
polygon_model = model(points_sampled)

In [ ]:
# visualize the polygon with points
plt.plot(
    points_sampled[:, 0],
    points_sampled[:, 1],
    "bo",
    markersize=2,
    label="Sampled Points",
)

# show the ground truth model
plt.plot(
    fov.boundary_wrapped()[:, 0],
    fov.boundary_wrapped()[:, 1],
    "g-",
    label="Ground Truth Polygon",
)

# show the polylidar model (close the shape with concat)
plt.plot(
    polygon_model.boundary_wrapped()[:, 0],
    polygon_model.boundary_wrapped()[:, 1],
    "r-",
    label="PolyLidar Model",
)
plt.legend()
plt.show()

## For later...using a lidar physics model

In [ ]:
class LidarSensorModel:
    def __init__(
        self,
        position: np.ndarray = np.array([0, 0, 1.6]),
        channels: int = 32,
        max_range: float = 70.0,
        az_resolution_deg: float = 0.1,
        az_fov_deg: Tuple[float, float] = [-180, 180],
        el_fov_deg: Tuple[float, float] = [-17.6, 2.4],
    ):
        """ "Lidar sensor laser propagation model"""
        # set needed attributes
        self.position = position
        self.max_range = max_range

        # specify the grid space of azimuth and elevations
        n_az = int((az_fov_deg[1] - az_fov_deg[0]) // az_resolution_deg)
        azs = math.pi / 180 * np.linspace(az_fov_deg[0], az_fov_deg[1], n_az)
        els = math.pi / 180 * np.linspace(el_fov_deg[0], el_fov_deg[1], channels)

        # transform the polar coordinates into a collection of unit vectors
        # https://math.stackexchange.com/questions/1150232/finding-the-unit-direction-vector-given-azimuth-and-elevation
        self._laser_az_el = np.array([[az, el] for az in azs for el in els])
        self._laser_vectors = np.array(
            [
                [
                    np.sin(az) * np.cos(el),
                    np.cos(az) * np.cos(el),
                    np.sin(el),
                ]
                for az, el in self._laser_az_el
            ]
        )

    def propagate(
        self, fov: Union[FovPolygon, None] = None, smoothing: float = 1.0
    ) -> np.ndarray:
        """Propagate laser points until they hit the ground plane

        Allow for a polygon constraint to be enforced. If the propagated
        points are beyond the boundary of the polygon, cluster them
        at the boundary with some possible noise.
        """
        # solve for when the z = 0.0
        rng_at_z_zero = -self.position[2] / self._laser_vectors[:, 2]
        idx_inf = rng_at_z_zero <= 1
        rng_at_z_zero[idx_inf] = np.inf

        # enforce the boundary constraint
        if fov is not None:
            # construct a spline approximation of the FOV model in polar
            spline_polar = fov.boundary_spline_polar(smoothing=smoothing)
            rng_boundary = spline_polar(self._laser_az_el[~idx_inf, 0])
            rng_at_z_zero[~idx_inf] = np.minimum(rng_at_z_zero[~idx_inf], rng_boundary)

        # enforce the max range constraint
        pts_valid = rng_at_z_zero < self.max_range

        # package up all points from spherical to cartesian
        pts_polar = np.concatenate(
            (rng_at_z_zero[pts_valid, None], self._laser_az_el[pts_valid, :1]), axis=1
        )
        pts_xy = np.array(
            [
                pts_polar[:, 0] * np.cos(pts_polar[:, 1]),
                pts_polar[:, 0] * np.sin(pts_polar[:, 1]),
            ]
        ).T

        return pts_xy

In [ ]:
# test an unimpeded lidar sensor model
lidar = LidarSensorModel(channels=32)
pts_xy = lidar.propagate()
plt.plot(pts_xy[:, 0], pts_xy[:, 1], "b.", markersize=1)
plt.show()

# test a lidar sensor model with a boundary
fov_test = random_fov(avg_radius=20)
lidar = LidarSensorModel(channels=32)
pts_xy = lidar.propagate(fov=fov_test, smoothing=0)
plt.plot(pts_xy[:, 0], pts_xy[:, 1], "g.", markersize=1)
plt.plot(
    fov_test.boundary_wrapped()[:, 0],
    fov_test.boundary_wrapped()[:, 1],
    "k-",
    label="Ground Truth Polygon",
)
plt.show()